In [ ]:
import pandas as pd
import numpy as np
import sys
import sklearn
import io
import random
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


train_url = '/content/drive/My Drive/IDS/NSL_KDD_Train.csv'
test_url = '/content/drive/My Drive/IDS/NSL_KDD_Test.csv'


In [ ]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]


df = pd.read_csv(train_url,header=None, names = col_names)

df_test = pd.read_csv(test_url, header=None, names = col_names)

print('Dimensions of the Training set:',df.shape)
print('Dimensions of the Test set:',df_test.shape)

In [ ]:
df.head(5)

In [ ]:
print('Label distribution Training set:')
print(df['label'].value_counts())
print()
print('Label distribution Test set:')
print(df_test['label'].value_counts())

In [ ]:
print('Training set:')
for col_name in df.columns:
    if df[col_name].dtypes == 'object' :
        unique_cat = len(df[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))

print()
print('Distribution of categories in service:')
print(df['service'].value_counts().sort_values(ascending=False).head())

In [ ]:
# Test set
print('Test set:')
for col_name in df_test.columns:
    if df_test[col_name].dtypes == 'object' :
        unique_cat = len(df_test[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))


**LabelEncoder**

**Insert categorical features into a 2D numpy array**

In [ ]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
categorical_columns=['protocol_type', 'service', 'flag']

df_categorical_values = df[categorical_columns]
testdf_categorical_values = df_test[categorical_columns]

df_categorical_values.head()

In [ ]:
# protocol type
unique_protocol=sorted(df.protocol_type.unique())
string1 = 'Protocol_type_'
unique_protocol2=[string1 + x for x in unique_protocol]
print(unique_protocol2)

# service
unique_service=sorted(df.service.unique())
string2 = 'service_'
unique_service2=[string2 + x for x in unique_service]
print(unique_service2)


# flag
unique_flag=sorted(df.flag.unique())
string3 = 'flag_'
unique_flag2=[string3 + x for x in unique_flag]
print(unique_flag2)


# put together
dumcols=unique_protocol2 + unique_service2 + unique_flag2


#do it for test set
unique_service_test=sorted(df_test.service.unique())
unique_service2_test=[string2 + x for x in unique_service_test]
testdumcols=unique_protocol2 + unique_service2_test + unique_flag2


**Transform categorical features into numbers using LabelEncoder()**

In [ ]:
df_categorical_values_enc=df_categorical_values.apply(LabelEncoder().fit_transform)

print(df_categorical_values.head())
print('--------------------')
print(df_categorical_values_enc.head())

# test set
testdf_categorical_values_enc=testdf_categorical_values.apply(LabelEncoder().fit_transform)

**One-Hot-Encoding**

In [ ]:
enc = OneHotEncoder(categories='auto')
df_categorical_values_encenc = enc.fit_transform(df_categorical_values_enc)
df_cat_data = pd.DataFrame(df_categorical_values_encenc.toarray(),columns=dumcols)


# test set
testdf_categorical_values_encenc = enc.fit_transform(testdf_categorical_values_enc)
testdf_cat_data = pd.DataFrame(testdf_categorical_values_encenc.toarray(),columns=testdumcols)

df_cat_data.head()

In [ ]:
trainservice=df['service'].tolist()
testservice= df_test['service'].tolist()
difference=list(set(trainservice) - set(testservice))
string = 'service_'
difference=[string + x for x in difference]
difference

In [ ]:
for col in difference:
    testdf_cat_data[col] = 0

print(df_cat_data.shape)
print(testdf_cat_data.shape)

In [ ]:
newdf=df.join(df_cat_data)
newdf.drop('flag', axis=1, inplace=True)
newdf.drop('protocol_type', axis=1, inplace=True)
newdf.drop('service', axis=1, inplace=True)

# test data
newdf_test=df_test.join(testdf_cat_data)
newdf_test.drop('flag', axis=1, inplace=True)
newdf_test.drop('protocol_type', axis=1, inplace=True)
newdf_test.drop('service', axis=1, inplace=True)

print(newdf.shape)
print(newdf_test.shape)

In [ ]:
labeldf=newdf['label']
labeldf_test=newdf_test['label']


# change the label column
newlabeldf=labeldf.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 1,'nmap' : 1,'portsweep' : 1,'satan' : 1,'mscan' : 1,'saint' : 1,
                            'ftp_write': 1,'guess_passwd': 1,'imap': 1,'multihop': 1,'phf': 1,'spy': 1,'warezclient': 1,'warezmaster': 1,'sendmail': 1,'named': 1,'snmpgetattack': 1,'snmpguess': 1,'xlock': 1,'xsnoop': 1,'httptunnel': 1,
                           'buffer_overflow': 1,'loadmodule': 1,'perl': 1,'rootkit': 1,'ps': 1,'sqlattack': 1,'xterm': 1 })
newlabeldf_test=labeldf_test.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 1,'nmap' : 1,'portsweep' : 1,'satan' : 1,'mscan' : 1,'saint' : 1
                           ,'ftp_write': 1,'guess_passwd': 1,'imap': 1,'multihop': 1,'phf': 1,'spy': 1,'warezclient': 1,'warezmaster': 1,'sendmail': 1,'named': 1,'snmpgetattack': 1,'snmpguess': 1,'xlock': 1,'xsnoop': 1,'httptunnel': 1,
                           'buffer_overflow': 1,'loadmodule': 1,'perl': 1,'rootkit': 1,'ps': 1,'sqlattack': 1,'xterm': 1})


# put the new label column back
newdf['label'] = newlabeldf
newdf_test['label'] = newlabeldf_test

In [ ]:
X_Df = newdf.drop('label',axis=1)
newdf.head(5)

**Step 2: Feature Scaling**

In [ ]:
Y_Df = newdf.label
X_Df_test = newdf.drop('label',axis=1)
Y_Df_test = newdf_test.label

In [ ]:
from keras.utils import to_categorical
y_binary = to_categorical(Y_Df)
y_test_binary = to_categorical(Y_Df_test)

In [ ]:
colNames=list(X_Df)
colNames_test=list(X_Df_test)

In [ ]:
X_Df

In [ ]:
X_Df_test

In [ ]:
from sklearn import preprocessing
scaler1 = preprocessing.StandardScaler().fit(X_Df)
X_Df=scaler1.transform(X_Df)

scaler5 = preprocessing.StandardScaler().fit(X_Df_test)
X_Df_test=scaler5.transform(X_Df_test)

In [ ]:
X_Df_test

array([[-0.11024922, -0.0076786 , -0.00491864, ..., -0.01972622,
         0.82515007, -0.04643159],
       [-0.11024922, -0.00773737, -0.00491864, ..., -0.01972622,
         0.82515007, -0.04643159],
       [-0.11024922, -0.00776224, -0.00491864, ..., -0.01972622,
        -1.21190076, -0.04643159],
       ...,
       [-0.11024922, -0.00738219, -0.00482315, ..., -0.01972622,
         0.82515007, -0.04643159],
       [-0.11024922, -0.00776224, -0.00491864, ..., -0.01972622,
        -1.21190076, -0.04643159],
       [-0.11024922, -0.00773652, -0.00491864, ..., -0.01972622,
         0.82515007, -0.04643159]])

In [ ]:
X_Df

array([[-0.11024922, -0.0076786 , -0.00491864, ..., -0.01972622,
         0.82515007, -0.04643159],
       [-0.11024922, -0.00773737, -0.00491864, ..., -0.01972622,
         0.82515007, -0.04643159],
       [-0.11024922, -0.00776224, -0.00491864, ..., -0.01972622,
        -1.21190076, -0.04643159],
       ...,
       [-0.11024922, -0.00738219, -0.00482315, ..., -0.01972622,
         0.82515007, -0.04643159],
       [-0.11024922, -0.00776224, -0.00491864, ..., -0.01972622,
        -1.21190076, -0.04643159],
       [-0.11024922, -0.00773652, -0.00491864, ..., -0.01972622,
         0.82515007, -0.04643159]])

In [ ]:
from keras.utils import to_categorical
y_binary = to_categorical(Y_Df)
y_test_binary = to_categorical(Y_Df_test)

#   

# Neural Network

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.datasets import mnist
from IPython.display import SVG
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import load_model
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

In [ ]:
model = Sequential()
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(10, kernel_initializer='normal'))
model.add(Dense(2, activation='softmax'))

In [ ]:
from tensorflow.keras.metrics import F1Score, Precision, Recall

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', F1Score(), Precision(), Recall()])


In [ ]:
# overfitting, underfitting
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=3, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath="model.keras", verbose=0, save_best_only=True)

In [ ]:
history = model.fit(X_Df, y_binary, batch_size=128, epochs=50, verbose=1, validation_split=0.1 ,callbacks=[monitor])

f1_val = history.history['val_f1_score'][-1]
precision_val = history.history['val_precision'][-1]
recall_val = history.history['val_recall'][-1]

print(f"F1 Score (Validation): {f1_val}")
print(f"Precision (Validation): {precision_val}")
print(f"Recall (Validation): {recall_val}")

Epoch 1/50
491/886 [===============>..............] - ETA: 5s - loss: 0.0539 - accuracy: 0.9829 - f1_score: 0.9828 - precision_1: 0.9829 - recall_1: 0.9829

In [ ]:
len(Y_Df_test)

In [ ]:
from sklearn.metrics import confusion_matrix

#y_pred = model.predict_classes(X_Df_test)  # Assuming multi-class problem (change to predict for binary)
y_pred = model.predict(X_Df_test)  # Assuming multi-class problem (change to predict for binary)
y_pred = np.argmax(y_pred, axis=1)
cm = confusion_matrix(Y_Df_test, y_pred)

#cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#cm_normalized = cm.astype('float') / 22544

print("Confusion Matrix:")
#print(cm_normalized)

print(cm)

target_names=["Normal", "DoS",  "Probe", "R2L", "U2R"]
cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(6,4))
sns.heatmap(cmn, annot=True, fmt='.2f', xticklabels=target_names, yticklabels=target_names,cmap='Blues')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show(block=False)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Plot of accuracy vs epoch for train and test dataset")
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Plot of loss vs epoch for train and test dataset")
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()